# Data Exploration

In this notebook, we take an in-depth look at the data and aim to provide insights that can help improve feature engineering. Please feel free to make edits or add your findings to this notebook.

__Highlights__

- Many columns have missing values.
  - For missing values in the `session` column, we could attempt to fill them using the `date` or `home_team_season` columns.
  - We can try to infer a missing stat by the team's past season performance. i.e. looking at the team's `home_batting_RBI_10RA` from previous games

- Task 2 data contains categorical values not present in the training data, e.g., Task 2 has different `home_pitcher` values.

- Some categorical columns may not be useful:
  - `home_team_season` and `away_team_season` are simply "team name + season year" (e.g., 'TBR_2019'), which is redundant information since we already have `session`, `home_team_abbr`, and `away_team_abbr`.

In [29]:
import pandas as pd
import tqdm

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('data/task1/train_data.csv')
df.head()

,id,home_team_abbr,away_team_abbr,date,is_night_game,home_team_win,home_pitcher,away_pitcher,home_team_rest,away_team_rest,home_pitcher_rest,away_pitcher_rest,season,home_batting_batting_avg_10RA,home_batting_onbase_perc_10RA,home_batting_onbase_plus_slugging_10RA,home_batting_leverage_index_avg_10RA,home_batting_RBI_10RA,away_batting_batting_avg_10RA,away_batting_onbase_perc_10RA,away_batting_onbase_plus_slugging_10RA,away_batting_leverage_index_avg_10RA,away_batting_RBI_10RA,home_pitching_earned_run_avg_10RA,home_pitching_SO_batters_faced_10RA,home_pitching_H_batters_faced_10RA,home_pitching_BB_batters_faced_10RA,away_pitching_earned_run_avg_10RA,away_pitching_SO_batters_faced_10RA,away_pitching_H_batters_faced_10RA,away_pitching_BB_batters_faced_10RA,home_pitcher_earned_run_avg_10RA,home_pitcher_SO_batters_faced_10RA,home_pitcher_H_batters_faced_10RA,home_pitcher_BB_batters_faced_10RA,away_pitcher_earned_run_avg_10RA,away_pitcher_SO_batters_faced_10RA,away_pitcher_H_batters_faced_10RA,away_pitcher_BB_batters_faced_10RA,home_team_season,away_team_season,home_team_errors_mean,home_team_errors_std,home_team_errors_skew,away_team_errors_mean,away_team_errors_std,away_team_errors_skew,home_team_spread_mean,home_team_spread_std,home_team_spread_skew,away_team_spread_mean,away_team_spread_std,away_team_spread_skew,home_team_wins_mean,home_team_wins_std,home_team_wins_skew,away_team_wins_mean,away_team_wins_std,away_team_wins_skew,home_batting_batting_avg_mean,home_batting_batting_avg_std,home_batting_batting_avg_skew,home_batting_onbase_perc_mean,home_batting_onbase_perc_std,home_batting_onbase_perc_skew,home_batting_onbase_plus_slugging_mean,home_batting_onbase_plus_slugging_std,home_batting_onbase_plus_slugging_skew,home_batting_leverage_index_avg_mean,home_batting_leverage_index_avg_std,home_batting_leverage_index_avg_skew,home_batting_wpa_bat_mean,home_batting_wpa_bat_std,home_batting_wpa_bat_skew,home_batting_RBI_mean,home_batting_RBI_std,home_batting_RBI_skew,away_batting_batting_avg_mean,away_batting_batting_avg_std,away_batting_batting_avg_skew,away_batting_onbase_perc_mean,away_batting_onbase_perc_std,away_batting_onbase_perc_skew,away_batting_onbase_plus_slugging_mean,away_batting_onbase_plus_slugging_std,away_batting_onbase_plus_slugging_skew,away_batting_leverage_index_avg_mean,away_batting_leverage_index_avg_std,away_batting_leverage_index_avg_skew,away_batting_wpa_bat_mean,away_batting_wpa_bat_std,away_batting_wpa_bat_skew,away_batting_RBI_mean,away_batting_RBI_std,away_batting_RBI_skew,home_pitching_earned_run_avg_mean,home_pitching_earned_run_avg_std,home_pitching_earned_run_avg_skew,home_pitching_SO_batters_faced_mean,home_pitching_SO_batters_faced_std,home_pitching_SO_batters_faced_skew,home_pitching_H_batters_faced_mean,home_pitching_H_batters_faced_std,home_pitching_H_batters_faced_skew,home_pitching_BB_batters_faced_mean,home_pitching_BB_batters_faced_std,home_pitching_BB_batters_faced_skew,home_pitching_leverage_index_avg_mean,home_pitching_leverage_index_avg_std,home_pitching_leverage_index_avg_skew,home_pitching_wpa_def_mean,home_pitching_wpa_def_std,home_pitching_wpa_def_skew,away_pitching_earned_run_avg_mean,away_pitching_earned_run_avg_std,away_pitching_earned_run_avg_skew,away_pitching_SO_batters_faced_mean,away_pitching_SO_batters_faced_std,away_pitching_SO_batters_faced_skew,away_pitching_H_batters_faced_mean,away_pitching_H_batters_faced_std,away_pitching_H_batters_faced_skew,away_pitching_BB_batters_faced_mean,away_pitching_BB_batters_faced_std,away_pitching_BB_batters_faced_skew,away_pitching_leverage_index_avg_mean,away_pitching_leverage_index_avg_std,away_pitching_leverage_index_avg_skew,away_pitching_wpa_def_mean,away_pitching_wpa_def_std,away_pitching_wpa_def_skew,home_pitcher_earned_run_avg_mean,home_pitcher_earned_run_avg_std,home_pitcher_earned_run_avg_skew,home_pitcher_SO_batters_faced_mean,home_pitcher_SO_batters_faced_std,home_pitcher_SO_batters_faced_skew,home_pitcher_H_batters_faced_mea

In [3]:
df.describe()

,id,home_team_rest,away_team_rest,home_pitcher_rest,away_pitcher_rest,season,home_batting_batting_avg_10RA,home_batting_onbase_perc_10RA,home_batting_onbase_plus_slugging_10RA,home_batting_leverage_index_avg_10RA,home_batting_RBI_10RA,away_batting_batting_avg_10RA,away_batting_onbase_perc_10RA,away_batting_onbase_plus_slugging_10RA,away_batting_leverage_index_avg_10RA,away_batting_RBI_10RA,home_pitching_earned_run_avg_10RA,home_pitching_SO_batters_faced_10RA,home_pitching_H_batters_faced_10RA,home_pitching_BB_batters_faced_10RA,away_pitching_earned_run_avg_10RA,away_pitching_SO_batters_faced_10RA,away_pitching_H_batters_faced_10RA,away_pitching_BB_batters_faced_10RA,home_pitcher_earned_run_avg_10RA,home_pitcher_SO_batters_faced_10RA,home_pitcher_H_batters_faced_10RA,home_pitcher_BB_batters_faced_10RA,away_pitcher_earned_run_avg_10RA,away_pitcher_SO_batters_faced_10RA,away_pitcher_H_batters_faced_10RA,away_pitcher_BB_batters_faced_10RA,home_team_errors_mean,home_team_errors_std,home_team_errors_skew,away_team_errors_mean,away_team_errors_std,away_team_errors_skew,home_team_spread_mean,home_team_spread_std,home_team_spread_skew,away_team_spread_mean,away_team_spread_std,away_team_spread_skew,home_team_wins_mean,home_team_wins_std,home_team_wins_skew,away_team_wins_mean,away_team_wins_std,away_team_wins_skew,home_batting_batting_avg_mean,home_batting_batting_avg_std,home_batting_batting_avg_skew,home_batting_onbase_perc_mean,home_batting_onbase_perc_std,home_batting_onbase_perc_skew,home_batting_onbase_plus_slugging_mean,home_batting_onbase_plus_slugging_std,home_batting_onbase_plus_slugging_skew,home_batting_leverage_index_avg_mean,home_batting_leverage_index_avg_std,home_batting_leverage_index_avg_skew,home_batting_wpa_bat_mean,home_batting_wpa_bat_std,home_batting_wpa_bat_skew,home_batting_RBI_mean,home_batting_RBI_std,home_batting_RBI_skew,away_batting_batting_avg_mean,away_batting_batting_avg_std,away_batting_batting_avg_skew,away_batting_onbase_perc_mean,away_batting_onbase_perc_std,away_batting_onbase_perc_skew,away_batting_onbase_plus_slugging_mean,away_batting_onbase_plus_slugging_std,away_batting_onbase_plus_slugging_skew,away_batting_leverage_index_avg_mean,away_batting_leverage_index_avg_std,away_batting_leverage_index_avg_skew,away_batting_wpa_bat_mean,away_batting_wpa_bat_std,away_batting_wpa_bat_skew,away_batting_RBI_mean,away_batting_RBI_std,away_batting_RBI_skew,home_pitching_earned_run_avg_mean,home_pitching_earned_run_avg_std,home_pitching_earned_run_avg_skew,home_pitching_SO_batters_faced_mean,home_pitching_SO_batters_faced_std,home_pitching_SO_batters_faced_skew,home_pitching_H_batters_faced_mean,home_pitching_H_batters_faced_std,home_pitching_H_batters_faced_skew,home_pitching_BB_batters_faced_mean,home_pitching_BB_batters_faced_std,home_pitching_BB_batters_faced_skew,home_pitching_leverage_index_avg_mean,home_pitching_leverage_index_avg_std,home_pitching_leverage_index_avg_skew,home_pitching_wpa_def_mean,home_pitching_wpa_def_std,home_pitching_wpa_def_skew,away_pitching_earned_run_avg_mean,away_pitching_earned_run_avg_std,away_pitching_earned_run_avg_skew,away_pitching_SO_batters_faced_mean,away_pitching_SO_batters_faced_std,away_pitching_SO_batters_faced_skew,away_pitching_H_batters_faced_mean,away_pitching_H_batters_faced_std,away_pitching_H_batters_faced_skew,away_pitching_BB_batters_faced_mean,away_pitching_BB_batters_faced_std,away_pitching_BB_batters_faced_skew,away_pitching_leverage_index_avg_mean,away_pitching_leverage_index_avg_std,away_pitching_leverage_index_avg_skew,away_pitching_wpa_def_mean,away_pitching_wpa_def_std,away_pitching_wpa_def_skew,home_pitcher_earned_run_avg_mean,home_pitcher_earned_run_avg_std,home_pitcher_earned_run_avg_skew,home_pitcher_SO_batters_faced_mean,home_pitcher_SO_batters_faced_std,home_pitcher_SO_batters_faced_skew,home_pitcher_H_batters_faced_mean,home_pitcher_H_batters_faced_std,home_pitcher_H_batters_faced_skew,home_pitcher_BB_batters_faced_mean,home_pitcher_BB_bat

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11067 entries, 0 to 11066
Columns: 167 entries, id to away_pitcher_wpa_def_skew
dtypes: bool(1), float64(157), int64(1), object(8)
memory usage: 14.0+ MB


Checking for missing values. It seems like majority of column has missing values.

In [5]:
missing_df = pd.DataFrame({
    'type': df.dtypes,
    'missing': df.isnull().any(),
    'missing count': df.isnull().sum(),
    'missing percentage': (df.isnull().sum() / len(df)) * 100
})
missing_df.sort_values(by='missing percentage', ascending=False, inplace=True)
pd.set_option('display.max_rows', None)
missing_df

,type,missing,missing count,missing percentage
home_pitcher_earned_run_avg_skew,float64,True,2692,24.324569
home_pitcher_BB_batters_faced_skew,float64,True,2685,24.261317
home_pitcher_H_batters_faced_skew,float64,True,2680,24.216138
home_pitcher_leverage_index_avg_skew,float64,True,2676,24.179995
home_pitcher_wpa_def_skew,float64,True,2673,24.152887
away_pitcher_SO_batters_faced_skew,float64,True,2672,24.143851
home_pitcher_SO_batters_faced_skew,float64,True,2672,24.143851
away_pitcher_BB_batters_faced_skew,float64,True,2660,24.035421
away_pitcher_wpa_def_skew,float64,True,2647,23.917954
away_pitcher_leverage_index_avg_skew,float64,True,2646,23.908918


## Dealing Missing Column

### Season
We can infer in season column by 'date', 'home_team_season' or 'away_team_season' column

In [7]:
missing = df[df['season'].isna()]
missing[['id', 'date', 'season', 'home_team_abbr', 'home_team_season', 'away_team_season']].head(5)

,id,date,season,home_team_abbr,home_team_season,away_team_season
18,18,2017-04-20,NaN,RAV,MIL_2017,STL_2017
36,36,2021-04-30,NaN,RKN,SEA_2021,LAA_2021
62,62,2019-07-26,NaN,SAJ,TOR_2019,TBR_2019
108,108,2021-06-18,NaN,JBM,KCR_2021,BOS_2021
130,130,2018-05-08,NaN,UPV,NYY_2018,BOS_2018


In [12]:
from datetime import datetime
import warnings



def fill_season(df):
    def fill(row):
        if not pd.isna(row['season']):
            return row['season']

        if isinstance(row['date'], str):
            d = datetime.strptime(row['date'], '%Y-%m-%d')
            return d.year

        if isinstance(row['home_team_season'], str):
            return int(row['home_team_season'][-4:])

        warnings.warn("unable to infer season. use mock season instead")
        return 1

    df['season'] = df.apply(fill, axis=1)
    

x = df.copy()
print("row with missing season:", len(x[x['season'].isna()]))
fill_season(x)
x[['id', 'date', 'season', 'home_team_abbr', 'home_team_season', 'away_team_season']].head(5)
print("row with missing season after fill:", len(x[x['season'].isna()]))


row with missing season: 553
row with missing season after fill: 0


### Team statistic

In [9]:
missing = df[df['home_batting_RBI_mean'].isna()]
missing[['id', 'date', 'home_batting_RBI_mean']].head(5)

,id,date,home_batting_RBI_mean
15,15,2017-04-06,NaN
51,51,2017-07-06,NaN
65,65,2018-03-29,NaN
110,110,2017-04-03,NaN
115,115,2018-06-24,NaN


In [ ]:
# fill in team' numeric statistic base on team's season performance
# - you should apply this AFTER fill in the missing season value
# - you should NOT apply this for pitcher stat (i.e. 'home_pitching_H_batters_faced_mean') 
#   as it should be infer differently
# - here we assume a team's home & away stat should be simialr
def fill_stat(df, key):
    def fill(row):
        if not pd.isna(row[key]):
            return row[key]

        team_col = 'away_team_abbr' if 'away_' in key else 'home_team_abbr'
        team_abbr = row[team_col]
        season = row['season']

        # Here we want to infer a team performance stat base on past season data. 
        # The same performance could be appear in both "home_" and "_away"
        # i.e. home_pitching_H_batters_faced_mean and away_pitching_H_batters_faced_mean
        # We would use both the 'home' and 'away' of version of stat and take the average
        stat_key = 'tmp_key'

        away_stat_key = key.replace('home_', 'away_')
        away_stat = df[(df['away_team_abbr'] == team_abbr) & 
                       (df[away_stat_key].notna())]
        away_stat.rename(columns={away_stat_key: stat_key}, inplace=True)
        
        home_stat_key = key.replace('away_', 'home_')
        home_stat = df[(df['home_team_abbr'] == team_abbr) & 
                       (df[home_stat_key].notna())]
        home_stat.rename(columns={home_stat_key: stat_key}, inplace=True)

        team_stat = pd.concat([away_stat, home_stat], axis=0)
        season_stat = team_stat[team_stat['season'] == season]

        if len(season_stat) > 0:
            return season_stat[stat_key].mean()

        # # TODO: not sure if cross season stat make sense.
        # if len(team_stat) > 0:
        #     return team_stat[stat_key].mean()
        return row[key]
# 
    df[key] = df.apply(fill, axis=1)
    missing_count = len(df[df[key].isna()])
    if missing_count > 0:
        warnings.warn(f"fill_stat: {key} column still has {missing_count} missing values")


x = df.copy()
fill_season(x)

key = "home_batting_RBI_mean"
print(f"row with missing {key}:", len(x[x[key].isna()]))
fill_stat(x, key)
print(f"row with missing {key} after:", len(x[x[key].isna()]))

row with missing home_batting_RBI_mean: 650
row with missing home_batting_RBI_mean after: 0


Here are list of the team performance statistic that we could apply fill_stat() with

In [32]:
float_col = list(df.select_dtypes(include='float').columns)
exclude = ['season', 'home_team_rest', 'away_team_rest']
team_stat_col = [_ for _ in float_col if 
                 'pitcher' not in _ and 
                 'pitching' not in _ and 
                 _ not in exclude]
team_stat_col

['home_batting_batting_avg_10RA',
 'home_batting_onbase_perc_10RA',
 'home_batting_onbase_plus_slugging_10RA',
 'home_batting_leverage_index_avg_10RA',
 'home_batting_RBI_10RA',
 'away_batting_batting_avg_10RA',
 'away_batting_onbase_perc_10RA',
 'away_batting_onbase_plus_slugging_10RA',
 'away_batting_leverage_index_avg_10RA',
 'away_batting_RBI_10RA',
 'home_team_errors_mean',
 'home_team_errors_std',
 'home_team_errors_skew',
 'away_team_errors_mean',
 'away_team_errors_std',
 'away_team_errors_skew',
 'home_team_spread_mean',
 'home_team_spread_std',
 'home_team_spread_skew',
 'away_team_spread_mean',
 'away_team_spread_std',
 'away_team_spread_skew',
 'home_team_wins_mean',
 'home_team_wins_std',
 'home_team_wins_skew',
 'away_team_wins_mean',
 'away_team_wins_std',
 'away_team_wins_skew',
 'home_batting_batting_avg_mean',
 'home_batting_batting_avg_std',
 'home_batting_batting_avg_skew',
 'home_batting_onbase_perc_mean',
 'home_batting_onbase_perc_std',
 'home_batting_onbase_perc

In [31]:
x = df.copy()
fill_season(x)
for col in tqdm.tqdm(team_stat_col):
    fill_stat(x, col)


100%|██████████| 67/67 [04:51<00:00,  4.34s/it]


## Task 2 dataset analysis

In [ ]:
task2_df = pd.read_csv('data/task2/2024_test_data.csv')
task2_df

,id,home_team_abbr,away_team_abbr,is_night_game,home_pitcher,away_pitcher,home_team_rest,away_team_rest,home_pitcher_rest,away_pitcher_rest,season,home_batting_batting_avg_10RA,home_batting_onbase_perc_10RA,home_batting_onbase_plus_slugging_10RA,home_batting_leverage_index_avg_10RA,home_batting_RBI_10RA,away_batting_batting_avg_10RA,away_batting_onbase_perc_10RA,away_batting_onbase_plus_slugging_10RA,away_batting_leverage_index_avg_10RA,away_batting_RBI_10RA,home_pitching_earned_run_avg_10RA,home_pitching_SO_batters_faced_10RA,home_pitching_H_batters_faced_10RA,home_pitching_BB_batters_faced_10RA,away_pitching_earned_run_avg_10RA,away_pitching_SO_batters_faced_10RA,away_pitching_H_batters_faced_10RA,away_pitching_BB_batters_faced_10RA,home_pitcher_earned_run_avg_10RA,home_pitcher_SO_batters_faced_10RA,home_pitcher_H_batters_faced_10RA,home_pitcher_BB_batters_faced_10RA,away_pitcher_earned_run_avg_10RA,away_pitcher_SO_batters_faced_10RA,away_pitcher_H_batters_faced_10RA,away_pitcher_BB_batters_faced_10RA,home_team_season,away_team_season,home_team_errors_mean,home_team_errors_std,home_team_errors_skew,away_team_errors_mean,away_team_errors_std,away_team_errors_skew,home_team_spread_mean,home_team_spread_std,home_team_spread_skew,away_team_spread_mean,away_team_spread_std,away_team_spread_skew,home_team_wins_mean,home_team_wins_std,home_team_wins_skew,away_team_wins_mean,away_team_wins_std,away_team_wins_skew,home_batting_batting_avg_mean,home_batting_batting_avg_std,home_batting_batting_avg_skew,home_batting_onbase_perc_mean,home_batting_onbase_perc_std,home_batting_onbase_perc_skew,home_batting_onbase_plus_slugging_mean,home_batting_onbase_plus_slugging_std,home_batting_onbase_plus_slugging_skew,home_batting_leverage_index_avg_mean,home_batting_leverage_index_avg_std,home_batting_leverage_index_avg_skew,home_batting_wpa_bat_mean,home_batting_wpa_bat_std,home_batting_wpa_bat_skew,home_batting_RBI_mean,home_batting_RBI_std,home_batting_RBI_skew,away_batting_batting_avg_mean,away_batting_batting_avg_std,away_batting_batting_avg_skew,away_batting_onbase_perc_mean,away_batting_onbase_perc_std,away_batting_onbase_perc_skew,away_batting_onbase_plus_slugging_mean,away_batting_onbase_plus_slugging_std,away_batting_onbase_plus_slugging_skew,away_batting_leverage_index_avg_mean,away_batting_leverage_index_avg_std,away_batting_leverage_index_avg_skew,away_batting_wpa_bat_mean,away_batting_wpa_bat_std,away_batting_wpa_bat_skew,away_batting_RBI_mean,away_batting_RBI_std,away_batting_RBI_skew,home_pitching_earned_run_avg_mean,home_pitching_earned_run_avg_std,home_pitching_earned_run_avg_skew,home_pitching_SO_batters_faced_mean,home_pitching_SO_batters_faced_std,home_pitching_SO_batters_faced_skew,home_pitching_H_batters_faced_mean,home_pitching_H_batters_faced_std,home_pitching_H_batters_faced_skew,home_pitching_BB_batters_faced_mean,home_pitching_BB_batters_faced_std,home_pitching_BB_batters_faced_skew,home_pitching_leverage_index_avg_mean,home_pitching_leverage_index_avg_std,home_pitching_leverage_index_avg_skew,home_pitching_wpa_def_mean,home_pitching_wpa_def_std,home_pitching_wpa_def_skew,away_pitching_earned_run_avg_mean,away_pitching_earned_run_avg_std,away_pitching_earned_run_avg_skew,away_pitching_SO_batters_faced_mean,away_pitching_SO_batters_faced_std,away_pitching_SO_batters_faced_skew,away_pitching_H_batters_faced_mean,away_pitching_H_batters_faced_std,away_pitching_H_batters_faced_skew,away_pitching_BB_batters_faced_mean,away_pitching_BB_batters_faced_std,away_pitching_BB_batters_faced_skew,away_pitching_leverage_index_avg_mean,away_pitching_leverage_index_avg_std,away_pitching_leverage_index_avg_skew,away_pitching_wpa_def_mean,away_pitching_wpa_def_std,away_pitching_wpa_def_skew,home_pitcher_earned_run_avg_mean,home_pitcher_earned_run_avg_std,home_pitcher_earned_run_avg_skew,home_pitcher_SO_batters_faced_mean,home_pitcher_SO_batters_faced_std,home_pitcher_SO_batters_faced_skew,home_pitcher_H_batters_faced_mean,home_pitcher_H_ba

Task 2 dataset contains games from 2024, while the train dataset contains games from 2016-2023. It is possible that the new session could contains team or players that do not appear in the previous session. Let check each categorial columns and see if the values shares

In [ ]:
train_cols_val = {col: set(task2_df[col].unique()) for col in task2_df.columns if task2_df[col].dtype == 'object'}
task2_cols_val = {col: set(df[col].unique()) for col in df.columns if df[col].dtype == 'object'}

In [31]:
train_cols_val.keys()

dict_keys(['home_team_abbr', 'away_team_abbr', 'is_night_game', 'home_pitcher', 'away_pitcher', 'home_team_season', 'away_team_season'])

In [ ]:
for col in train_cols_val.keys():
    print(col, train_cols_val[col] ^ task2_cols_val[col])

home_team_abbr set()
away_team_abbr set()
is_night_game set()
home_pitcher {'mcderch01', 'musgrjo01', 'boscawi01', 'cobbal01', 'imanash01', 'hollojo01', 'schulja02', 'fulmeca01', 'wielajo01', 'yarbrry01', 'felizmi01', 'geedi01', 'shielja02', 'latosma01', 'espinpa01', 'bernabr01', 'carasma01', 'garream01', 'gallayo01', 'hollade01', 'harriky01', 'reedco01', 'irvinja01', 'scottta02', 'scottch01', 'nanceto01', 'greenco01', 'oberhbr01', 'trivilo01', 'volstch01', 'hahnje01', 'gsellro01', 'mccauda01', 'junisja01', 'soriajo01', 'andersh01', 'armstsh01', 'owenshe02', 'tetreja01', 'bowmama01', 'koehlto01', 'parsowe01', 'kennebr02', 'ginnjt01', 'alberma01', 'roberni01', 'cavalca01', 'aquinja01', 'pivetni01', 'hernada02', 'sprinje01', 'myersto01', 'alzolad01', 'webblo01', 'wrighmi01', 'ynoahu01', 'rockeku01', 'iwakuhi01', 'oakstr01', 'liriafr01', 'gibsoky01', 'gonzagi01', 'martico02', 'allenlo01', 'tarnofr01', 'mchugco01', 'phelpda01', 'warread01', 'kleinph01', 'ponceco01', 'woodfja01', 'gilbelo01